# Modelado

##### Integrantes: 
##### - Harold Aquino Curisinche 
##### - Rosa Estrada Estrada
##### Descripción: Limpieza y transformación del dataset de reclamos de SUSALUD

### Importacion de librerias

In [30]:
import pandas as pd
import numpy as np

In [136]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [34]:
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

### Carga de datos

In [36]:
df = pd.read_csv("../data/processed/reclamos_clean.csv", low_memory=False)

In [37]:
df.head()

,ID_PERIODO,DE_TIPO_INSTITUCION,DE_MEDIO_PRESENTACION,DE_MEDIO_RECEPCION,FE_PRESEN_RECLA,DE_SERVICIO,DE_COMPETENCIA,DE_CLASIF_1,DE_ESTADO_RECLAMO,DE_ETAPA_RECLAMO,DESCRIPCION,MES,DIA_SEMANA,AÑO,DIA_MES,ES_FIN_SEMANA,AÑO_ID_PERIODO,MES_ID_PERIODO
0,202101,IAFAS,Físico,Libro de Reclamaciones Físico,2020-11-27,Desconocido,Si,Negar o demora en otorgar la cobertura en salud,Resuelto,Resultado y Notificación,Sin descripción,11,4,2020,27,0,2021,1
1,202101,IAFAS,Físico,Libro de Reclamaciones Físico,2021-01-04,Desconocido,Si,Cobrar indebidamente,Resuelto,Resultado y Notificación,Me cobraron por una vacuna que estaba incluida...,1,0,2021,4,0,2021,1
2,202101,IAFAS,Físico,Libro de Reclamaciones Físico,2021-01-06,Desconocido,Si,Otros relativos a las IAFAS,Concluido,Archivo y Custodia del Expediente,Las ambulancias de EsSalud no llegaron a tiemp...,1,2,2021,6,0,2021,1
3,202101,IAFAS,Físico,Libro de Reclamaciones Físico,2021-01-06,Desconocido,Si,Otros relativos a las IAFAS,Concluido,Archivo y Custodia del Expediente,La atención que brindan las IAFAS no ha sido d...,1,2,2021,6,0,2021,1
4,202101,IAFAS,Físico,Libro de Reclamaciones Físico,2021-01-08,Desconocido,Si,Otros relativos a las IAFAS,En trámite,Evaluación e investigación,He tenido problemas para acceder a mis servici...,1,4,2021,8,0,2021,1


### Variables categoricas

In [79]:
categorical_cols_onehot = ['DE_MEDIO_PRESENTACION', 'DE_MEDIO_RECEPCION', 'DE_ESTADO_RECLAMO', 'DE_ETAPA_RECLAMO', 'DE_SERVICIO']

In [81]:
categorical_cols_ordinal = ['DE_TIPO_INSTITUCION', 'DE_COMPETENCIA']

In [83]:
text_col = 'DESCRIPCION'

#### Datos de entrenamietno y de prueba

In [43]:
X = df[[
    'DE_TIPO_INSTITUCION', 'DE_MEDIO_PRESENTACION', 'DE_MEDIO_RECEPCION',
    'DE_SERVICIO', 'DE_COMPETENCIA', 'DE_ESTADO_RECLAMO', 'DE_ETAPA_RECLAMO',
    'DESCRIPCION', 'DIA_MES', 'MES', 'DIA_SEMANA', 'ES_FIN_SEMANA',
    'AÑO_ID_PERIODO', 'MES_ID_PERIODO'
]]

In [44]:
y = df['DE_CLASIF_1']

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [65]:
print(f"Tamaño entrenamiento: {X_train.shape}")
print(f"Tamaño prueba: {X_test.shape}")

Tamaño entrenamiento: (130111, 14)
Tamaño prueba: (32528, 14)


##### StopWords

In [9]:
import nltk

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from nltk.corpus import stopwords

In [12]:
stop_words_spanish = stopwords.words('spanish')

### Construccion del preprocesamiento

#### Creación los transformadores para la codificación

In [87]:
transformadores = []

## OneHot Encoder

In [89]:
if categorical_cols_onehot:
    transformadores.append(('onehot', Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ]), categorical_cols_onehot))

## Ordinal Encoder

In [91]:
if categorical_cols_ordinal:
    transformadores.append(('ordinal', Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
    ]), categorical_cols_ordinal))

## TfidfVectorizer

In [93]:
if text_col:
    transformadores.append(('text', TfidfVectorizer(max_features=5000), text_col))

In [95]:
preprocessor = ColumnTransformer(transformers=transformadores, remainder='passthrough')

In [110]:
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_cols_onehot),
#         ('ordinal', OrdinalEncoder(), categorical_cols_ordinal),
#         ('text', TfidfVectorizer(max_features=5000), text_col)
#         #('text', TfidfVectorizer(stop_words=stop_words_spanish, max_features=3000, ngram_range=(1,2), min_df=5), text_col)
#     ],
#     remainder='passthrough'
# )

#### Definimos el modelo completo (Random Forest Classifier)

In [97]:
#class_weight='balanced'
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

In [99]:
model.fit(X_train, y_train)

C:\Users\harol\anaconda3\Lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['DE_MEDIO_PRESENTACION',
                                                   'DE_MEDIO_RECEPCION',
                                                   'DE_ESTADO_RECLAMO',
                                                   'DE_ETAPA_RECLAMO',
                                                   'DE_SERVICIO']),
                                                 ('ordinal',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  ['DE_TIPO_INSTITUCION',
                                                   'DE_COMPETENCIA']),
                                                 ('text',
                                                  TfidfVectorizer(max_features=5000),
                                                  'DESCRIPCION')])),
                ('classifier', RandomForestClassifier(random_state=42))])

## Evaluación

In [100]:
print("Accuracy en Train:", model.score(X_train, y_train))
print("Accuracy en Test:", model.score(X_test, y_test))

Accuracy en Train: 0.9974329610870718
Accuracy en Test: 0.9546237088047221


In [101]:
y_pred = model.predict(X_test)

In [102]:
print(classification_report(y_test, y_pred))

                                                                                  precision    recall  f1-score   support

                                                                               -       0.99      1.00      0.99     12654
                                                            Cobrar indebidamente       1.00      1.00      1.00      2119
                      Demorar la gestión de la carta de garantía y/o reembolsos.       0.72      0.61      0.66      1055
                                Negar  o demora en otorgar la cobertura en salud       0.69      0.64      0.67      1821
                       Negar atención para el trámite de registro o acreditación       1.00      1.00      1.00       189
                     Negar el otorgamiento de prestaciones económicas o sociales       1.00      0.94      0.97        47
                                     Negar la acreditación de usuario asegurado.       0.76      0.81      0.79      2121
                       

#### Matriz de confusión

In [127]:
conf_matrix = confusion_matrix(y_test, y_pred_test)
print("\nMatriz de Confusión en Test:\n", conf_matrix)


Matriz de Confusión en Test:
 [[12645     0     0     9     0     0     0     0     0     0     0     0
      0     0     0     0     0     0]
 [    0  2119     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0]
 [   20     0   641   240     0     0   154     0     0     0     0     0
      0     0     0     0     0     0]
 [  116     0   145  1172     0     0   388     0     0     0     0     0
      0     0     0     0     0     0]
 [    0     0     0     0   189     0     0     0     0     0     0     0
      0     0     0     0     0     0]
 [    0     0     0     3     0    44     0     0     0     0     0     0
      0     0     0     0     0     0]
 [   10     0   110   274     0     0  1727     0     0     0     0     0
      0     0     0     0     0     0]
 [    0     0     0     0     0     0     0    61     0     0     0     0
      0     0     0     0     0     0]
 [    1     0     0     0     0     0     0     0     7     0    

### Validacion Cruzada

In [133]:
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print("\nAccuracy promedio en 5-fold CV:", scores.mean())


Accuracy promedio en 5-fold CV: 0.9309574983836878


## Guardado del modelo

In [143]:
import joblib

In [147]:
joblib.dump(model, '../models/modelo_clasificador.pkl')

['../models/modelo_clasificador.pkl']